# Eaternity API

This is the Eaternity API that can return the carbon impact of the food product/ ingrediant 

In [98]:
import requests
import json
import pandas as pd
import math
import re
from requests.auth import HTTPBasicAuth

YOUR_KEY = "SR012UWDjBT7fgivtbNZSp1IJylocm5a"
AUTH = HTTPBasicAuth(YOUR_KEY, "")
BASE_URL = "https://co2.eaternity.ch"

if YOUR_KEY == "CHANGEME":
    raise RuntimeError("Please change your api key!!")

def create_kitchen(name, kitchen_id, location):
    url = f"{BASE_URL}/api/kitchens/{kitchen_id}"

    body = {
        "kitchen": {
            "name": name,
            "location": location
        }
    }

    response = requests.put(url, json=body, auth=AUTH)
    if response.status_code not in [200, 201, 202]:
        print(f"ERROR: Failed PUTting kitchen {kitchen_id} with status {response.status_code}: '{response.text}'")
    else:
        print(f"SUCCESS: PUT kitchen {kitchen_id}")
        return response.json()

    
def put_recipe(recipe_id, recipe_name, kitchen_id, ingredients, bad_ingr):
    url = f"{BASE_URL}/api/kitchens/{kitchen_id}/recipes/{recipe_id}"

    ingredient_list = []
    for ingredient in ingredients.keys():
        if ingredient not in bad_ingr:
            print(ingredient)
            ingred_dict = {
                "id": ingredient,
                "names": [{"language": "en", "value": ingredient}],
                "amount": ingredients[ingredient],
                "unit": "gram"
            }
            ingredient_list.append(ingred_dict)
        
    body = {
        "recipe": {
            "titles": [
                {
                    "language": "en",
                    "value": recipe_name
                }
            ],
            "date": "2020-09-19",
            "location": "Schweiz",
            "servings": 1,
            "ingredients": ingredient_list
        }
    }

    response = requests.put(url, json=body, auth=AUTH)
    
    if response.status_code in [601, 602]:
        print(response.text)
        bad_ingredient = re.findall(r"FoodService(.*)\\", response.text)[0]
        bad_ingr.append(bad_ingredient.lower().strip())
        print(bad_ingr)
        return put_recipe(recipe_id, recipe_name, kitchen_id, ingredients, bad_ingr)
    
    elif response.status_code not in [200, 201, 202]:
        print(f"ERROR: Failed PUTting recipe {recipe_id} with status {response.status_code}: '{response.text}'")
    else:
        print(f"SUCCESS: PUT recipe {recipe_id}")
        print(response.json())
        print(bad_ingr)
        return response.json()


if __name__ == '__main__':
    kitchen_id = "our_name_id"
    create_kitchen("My First Kitchen", kitchen_id, "Switzerland")
    
    xls = pd.ExcelFile('SwissRe Sourcing Data.xlsx')
    recipes = pd.read_excel(xls, 'Menus', header=1)
    
    vegan_dict = {}
    vegetarian_dict = {}
    meat_dict = {}
    
    for index, row in recipes.iterrows():
        recipe = {}
        rowiter = 0
        for rowiter in range(8, 34, 2):
            if not pd.isnull(row[rowiter]):
                recipe[row[rowiter+1].lower().strip()] = row[rowiter]
        
        if row[5] == 'x': #Checking if vegan
            vegan_dict[row['Name']] = recipe
        elif row[6] == 'x': #Checking if vegan
            vegetarian_dict[row['Name']] = recipe
        else:
            meat_dict[row['Name']] = recipe
    
    #Create a list of ingredients that return errors from the eaternity
    #write carbon dictionary to file
    bad_ingr = []
    vegan_carbon_list = []
    for index, key in enumerate(vegan_dict):
        recipe_return = put_recipe(f'{index}_vegan', key, kitchen_id, vegan_dict[key], bad_ingr)
        print(recipe_return)
        vegan_carbon_list.append(recipe_return)
    
    with open('vegan.json', 'w') as fp:
        json.dump(vegan_carbon_list, fp)
        print("printed vegan")
    
    vegetarian_carbon_list = []
    for index, key in enumerate(vegetarian_dict):
        recipe_return = put_recipe(f'{index}veg', key, kitchen_id, vegetarian_dict[key], bad_ingr)
        print(recipe_return)
        vegetarian_carbon_list.append(recipe_return)
    
    with open('vegetarian.json', 'w') as fp:
        json.dump(vegetarian_carbon_list, fp)
    
    meat_carbon_list = []
    for index, key in enumerate(meat_dict):
        recipe_return = put_recipe(f'{index}_meat', key, kitchen_id, meat_dict[key], bad_ingr)
        print(recipe_return)
        meat_carbon_list.append(recipe_return)
    
    with open('meat.json', 'w') as fp:
        json.dump(meat_carbon_list, fp)
        
    

SUCCESS: PUT kitchen our_name_id
spaghetti
canola oil
onions
tomatoes
"Linking Product with id \"FoodServiceCanola oil\" is not yet matched. Retry next day."
['canola oil']
spaghetti
onions
tomatoes
SUCCESS: PUT recipe 0_vegan
{'statuscode': 200, 'message': '', 'request-id': 0, 'recipe-id': '0_vegan', 'recipe': {'id': '0_vegan', 'kitchen-id': 'our_name_id', 'co2-value': 423, 'eaternity-award': True, 'rating': 'A', 'titles': [{'language': 'en', 'value': 'Pasta'}], 'date': '2020-09-19', 'location': 'Schweiz', 'servings': 1, 'recipe-portions': 1, 'ingredients': [], 'food-unit': 0.8453666242424243}}
['canola oil']
{'statuscode': 200, 'message': '', 'request-id': 0, 'recipe-id': '0_vegan', 'recipe': {'id': '0_vegan', 'kitchen-id': 'our_name_id', 'co2-value': 423, 'eaternity-award': True, 'rating': 'A', 'titles': [{'language': 'en', 'value': 'Pasta'}], 'date': '2020-09-19', 'location': 'Schweiz', 'servings': 1, 'recipe-portions': 1, 'ingredients': [], 'food-unit': 0.8453666242424243}}
bread


SUCCESS: PUT recipe 8_vegan
{'statuscode': 200, 'message': '', 'request-id': 0, 'recipe-id': '8_vegan', 'recipe': {'id': '8_vegan', 'kitchen-id': 'our_name_id', 'co2-value': 405, 'eaternity-award': True, 'rating': 'A', 'titles': [{'language': 'en', 'value': 'Crispy Organic Beetroot and Miso'}], 'date': '2020-09-19', 'location': 'Schweiz', 'servings': 1, 'recipe-portions': 1, 'ingredients': [], 'food-unit': 0.7228062071212109}}
['canola oil', 'red bell pepper', 'parsely', 'tumeric', 'peppers', 'rosmary', 'red bell peppers']
{'statuscode': 200, 'message': '', 'request-id': 0, 'recipe-id': '8_vegan', 'recipe': {'id': '8_vegan', 'kitchen-id': 'our_name_id', 'co2-value': 405, 'eaternity-award': True, 'rating': 'A', 'titles': [{'language': 'en', 'value': 'Crispy Organic Beetroot and Miso'}], 'date': '2020-09-19', 'location': 'Schweiz', 'servings': 1, 'recipe-portions': 1, 'ingredients': [], 'food-unit': 0.7228062071212109}}
chickpeas
broth
coconut milk
pumpkin
cauliflower
spinach
poppadoms
"

"Linking Product with id \"FoodServicebalsamica\" is not yet matched. Retry next day."
['canola oil', 'red bell pepper', 'parsely', 'tumeric', 'peppers', 'rosmary', 'red bell peppers', 'poppadoms', 'mushroom', 'honeydew melon', 'balsamica']
lentils
lettuce
papaya
oats
peanuts
peach
olive oil
SUCCESS: PUT recipe 16_vegan
{'statuscode': 200, 'message': '', 'request-id': 0, 'recipe-id': '16_vegan', 'recipe': {'id': '16_vegan', 'kitchen-id': 'our_name_id', 'co2-value': 1053, 'eaternity-award': False, 'rating': 'B', 'titles': [{'language': 'en', 'value': 'Salad with papaya and peach'}], 'date': '2020-09-19', 'location': 'Schweiz', 'servings': 1, 'recipe-portions': 1, 'ingredients': [], 'food-unit': 0.7747827090909091}}
['canola oil', 'red bell pepper', 'parsely', 'tumeric', 'peppers', 'rosmary', 'red bell peppers', 'poppadoms', 'mushroom', 'honeydew melon', 'balsamica']
{'statuscode': 200, 'message': '', 'request-id': 0, 'recipe-id': '16_vegan', 'recipe': {'id': '16_vegan', 'kitchen-id': 'o

"Linking Product with id \"FoodServiceHalloumi \" is not yet matched. Retry next day."
['canola oil', 'red bell pepper', 'parsely', 'tumeric', 'peppers', 'rosmary', 'red bell peppers', 'poppadoms', 'mushroom', 'honeydew melon', 'balsamica', 'chantarelle mushrooms', 'cannellini beans', 'leak', 'halloumi']
balsamico
olive oil
zucchini
beetroot
savoy cabbage
SUCCESS: PUT recipe 4veg
{'statuscode': 200, 'message': '', 'request-id': 0, 'recipe-id': '4veg', 'recipe': {'id': '4veg', 'kitchen-id': 'our_name_id', 'co2-value': 570, 'eaternity-award': True, 'rating': 'A', 'titles': [{'language': 'en', 'value': 'Fried Halloumi with vegetables'}], 'date': '2020-09-19', 'location': 'Schweiz', 'servings': 1, 'recipe-portions': 1, 'ingredients': [], 'food-unit': 0.2915908370131696}}
['canola oil', 'red bell pepper', 'parsely', 'tumeric', 'peppers', 'rosmary', 'red bell peppers', 'poppadoms', 'mushroom', 'honeydew melon', 'balsamica', 'chantarelle mushrooms', 'cannellini beans', 'leak', 'halloumi']
{'s

SUCCESS: PUT recipe 10veg
{'statuscode': 200, 'message': '', 'request-id': 0, 'recipe-id': '10veg', 'recipe': {'id': '10veg', 'kitchen-id': 'our_name_id', 'co2-value': 613, 'eaternity-award': True, 'rating': 'A', 'titles': [{'language': 'en', 'value': 'Vegetable soup with chili, soy and shiitake muschrooms'}], 'date': '2020-09-19', 'location': 'Schweiz', 'servings': 1, 'recipe-portions': 1, 'ingredients': [], 'food-unit': 0.16170752060606058}}
['canola oil', 'red bell pepper', 'parsely', 'tumeric', 'peppers', 'rosmary', 'red bell peppers', 'poppadoms', 'mushroom', 'honeydew melon', 'balsamica', 'chantarelle mushrooms', 'cannellini beans', 'leak', 'halloumi', 'curry spice', 'parmesan cheese', 'feta cheese', 'shiitake mushroom']
{'statuscode': 200, 'message': '', 'request-id': 0, 'recipe-id': '10veg', 'recipe': {'id': '10veg', 'kitchen-id': 'our_name_id', 'co2-value': 613, 'eaternity-award': True, 'rating': 'A', 'titles': [{'language': 'en', 'value': 'Vegetable soup with chili, soy and s

...

SUCCESS: PUT recipe 44_meat
{'statuscode': 200, 'message': '', 'request-id': 0, 'recipe-id': '44_meat', 'recipe': {'id': '44_meat', 'kitchen-id': 'our_name_id', 'co2-value': 2464, 'eaternity-award': False, 'rating': 'C', 'titles': [{'language': 'en', 'value': 'Pan-fried noodles with shrimps'}], 'date': '2020-09-19', 'location': 'Schweiz', 'servings': 1, 'recipe-portions': 1, 'ingredients': [], 'food-unit': 0.7393123169696969}}
['canola oil', 'red bell pepper', 'parsely', 'tumeric', 'peppers', 'rosmary', 'red bell peppers', 'poppadoms', 'mushroom', 'honeydew melon', 'balsamica', 'chantarelle mushrooms', 'cannellini beans', 'leak', 'halloumi', 'curry spice', 'parmesan cheese', 'feta cheese', 'shiitake mushroom', 'pear', 'orechietti', 'pumpkin squash', 'pumpkin seed oil', 'brie cheese', 'porcini mushrooms', 'rocket salad', 'red onion', 'curry powder', 'rigatoni', 'tomoto sugo', 'apple sause', 'anchovy', 'mozarella cheese', 'china cabbage', 'ricotta chease', 'carnaroli rice', 'fish sticks'

"Linking Product with id \"FoodServicesage leaves\" is not yet matched. Retry next day."
['canola oil', 'red bell pepper', 'parsely', 'tumeric', 'peppers', 'rosmary', 'red bell peppers', 'poppadoms', 'mushroom', 'honeydew melon', 'balsamica', 'chantarelle mushrooms', 'cannellini beans', 'leak', 'halloumi', 'curry spice', 'parmesan cheese', 'feta cheese', 'shiitake mushroom', 'pear', 'orechietti', 'pumpkin squash', 'pumpkin seed oil', 'brie cheese', 'porcini mushrooms', 'rocket salad', 'red onion', 'curry powder', 'rigatoni', 'tomoto sugo', 'apple sause', 'anchovy', 'mozarella cheese', 'china cabbage', 'ricotta chease', 'carnaroli rice', 'fish sticks', 'brocolli', 'canteloupe', 'swiss chard', 'swiss salmon', 'beef sausage', 'fried onions', 'poatatoes', 'hoisin sauce', 'rice wine vinegar', 'brown sugar', 'cornstarch', 'udon noodles', 'fish fillet', 'breadcrumbs', 'prawns', 'chicken breast', 'pastrami', 'semolina', 'brussel sprouts', 'ricotta chese', 'macaroni', 'pork sausage', 'wheat whe

SUCCESS: PUT recipe 51_meat
{'statuscode': 200, 'message': '', 'request-id': 0, 'recipe-id': '51_meat', 'recipe': {'id': '51_meat', 'kitchen-id': 'our_name_id', 'co2-value': 1460, 'eaternity-award': False, 'rating': 'C', 'titles': [{'language': 'en', 'value': 'Pork with spinach risotto'}], 'date': '2020-09-19', 'location': 'Schweiz', 'servings': 1, 'recipe-portions': 1, 'ingredients': [], 'food-unit': 2.054432336969697}}
['canola oil', 'red bell pepper', 'parsely', 'tumeric', 'peppers', 'rosmary', 'red bell peppers', 'poppadoms', 'mushroom', 'honeydew melon', 'balsamica', 'chantarelle mushrooms', 'cannellini beans', 'leak', 'halloumi', 'curry spice', 'parmesan cheese', 'feta cheese', 'shiitake mushroom', 'pear', 'orechietti', 'pumpkin squash', 'pumpkin seed oil', 'brie cheese', 'porcini mushrooms', 'rocket salad', 'red onion', 'curry powder', 'rigatoni', 'tomoto sugo', 'apple sause', 'anchovy', 'mozarella cheese', 'china cabbage', 'ricotta chease', 'carnaroli rice', 'fish sticks', 'bro

SUCCESS: PUT recipe 56_meat
{'statuscode': 200, 'message': '', 'request-id': 0, 'recipe-id': '56_meat', 'recipe': {'id': '56_meat', 'kitchen-id': 'our_name_id', 'co2-value': 939, 'eaternity-award': False, 'rating': 'B', 'titles': [{'language': 'en', 'value': 'Chicken piccata with spaghetti'}], 'date': '2020-09-19', 'location': 'Schweiz', 'servings': 1, 'recipe-portions': 1, 'ingredients': [], 'food-unit': 2.1619463042424245}}
['canola oil', 'red bell pepper', 'parsely', 'tumeric', 'peppers', 'rosmary', 'red bell peppers', 'poppadoms', 'mushroom', 'honeydew melon', 'balsamica', 'chantarelle mushrooms', 'cannellini beans', 'leak', 'halloumi', 'curry spice', 'parmesan cheese', 'feta cheese', 'shiitake mushroom', 'pear', 'orechietti', 'pumpkin squash', 'pumpkin seed oil', 'brie cheese', 'porcini mushrooms', 'rocket salad', 'red onion', 'curry powder', 'rigatoni', 'tomoto sugo', 'apple sause', 'anchovy', 'mozarella cheese', 'china cabbage', 'ricotta chease', 'carnaroli rice', 'fish sticks',

SUCCESS: PUT recipe 60_meat
{'statuscode': 200, 'message': '', 'request-id': 0, 'recipe-id': '60_meat', 'recipe': {'id': '60_meat', 'kitchen-id': 'our_name_id', 'co2-value': 4169, 'eaternity-award': False, 'rating': 'D', 'titles': [{'language': 'en', 'value': 'Cevapcici with ajvar'}], 'date': '2020-09-19', 'location': 'Schweiz', 'servings': 1, 'recipe-portions': 1, 'ingredients': [], 'food-unit': 1.2372270606060605}}
['canola oil', 'red bell pepper', 'parsely', 'tumeric', 'peppers', 'rosmary', 'red bell peppers', 'poppadoms', 'mushroom', 'honeydew melon', 'balsamica', 'chantarelle mushrooms', 'cannellini beans', 'leak', 'halloumi', 'curry spice', 'parmesan cheese', 'feta cheese', 'shiitake mushroom', 'pear', 'orechietti', 'pumpkin squash', 'pumpkin seed oil', 'brie cheese', 'porcini mushrooms', 'rocket salad', 'red onion', 'curry powder', 'rigatoni', 'tomoto sugo', 'apple sause', 'anchovy', 'mozarella cheese', 'china cabbage', 'ricotta chease', 'carnaroli rice', 'fish sticks', 'brocoll

In [93]:
meat_carbon_list

[None,
 None,
 None,
 {'statuscode': 200,
  'message': '',
  'request-id': 0,
  'recipe-id': '3',
  'recipe': {'id': '3',
   'kitchen-id': 'our_name_id',
   'co2-value': 644,
   'eaternity-award': False,
   'rating': 'B',
   'titles': [{'language': 'en', 'value': 'Chicken Satay Sandwich'}],
   'date': '2020-09-19',
   'location': 'Schweiz',
   'servings': 1,
   'recipe-portions': 1,
   'ingredients': [],
   'food-unit': 2.0225808387878788}},
 None,
 None,
 None,
 {'statuscode': 200,
  'message': '',
  'request-id': 0,
  'recipe-id': '7',
  'recipe': {'id': '7',
   'kitchen-id': 'our_name_id',
   'co2-value': 5542,
   'eaternity-award': False,
   'rating': 'E',
   'titles': [{'language': 'en', 'value': 'Veal'}],
   'date': '2020-09-19',
   'location': 'Schweiz',
   'servings': 1,
   'recipe-portions': 1,
   'ingredients': [],
   'food-unit': 1.762464253030303}},
 None,
 None,
 None,
 {'statuscode': 200,
  'message': '',
  'request-id': 0,
  'recipe-id': '11',
  'recipe': {'id': '11',
  

In [33]:
vegetarian = recipes.loc[recipes['Vegetarian?'] == 'x']

In [34]:
vegetarian.head()

,Menu + Option,Unnamed: 1,Name,Lactose free?,Gluten Free?,Vegan?,Vegetarian?,Grab and Go?,Amount g,Product 1,...,Amount g.8,Product 9,Amount g.9,Product 10,Amount g.10,Product 11,Amount g.11,Product 12,Amount g.12,Product 13
0,Menu 1,Option 1,Chantarelle Sour Cream,NaN,x,NaN,x,x,150.0,Chantarelle mushrooms,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,Option 3,Pasta,NaN,NaN,x,x,NaN,120.0,Spaghetti,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,Option 6,Vegi Sandwich,NaN,NaN,x,x,x,200.0,bread,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Menu 2,Option 1,Tomato soup,NaN,x,x,x,x,150.0,tomatoes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,NaN,Option 4,Shakshuka,x,x,NaN,x,NaN,200.0,tomatoes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
vegan = recipes.loc[recipes['Vegan?'] == 'x']

In [36]:
vegan.head()

,Menu + Option,Unnamed: 1,Name,Lactose free?,Gluten Free?,Vegan?,Vegetarian?,Grab and Go?,Amount g,Product 1,...,Amount g.8,Product 9,Amount g.9,Product 10,Amount g.10,Product 11,Amount g.11,Product 12,Amount g.12,Product 13
2,NaN,Option 3,Pasta,NaN,NaN,x,x,NaN,120.0,Spaghetti,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,Option 6,Vegi Sandwich,NaN,NaN,x,x,x,200.0,bread,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Menu 2,Option 1,Tomato soup,NaN,x,x,x,x,150.0,tomatoes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19,Menu 4,Option 1,Lentil and coconut soup,x,x,x,x,x,40.0,red lentils,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22,NaN,Option 4,Peppers stuffed with tofu,x,x,x,x,x,150.0,Peppers,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
meat = recipes.loc[recipes['Vegetarian?'] != 'x']

In [38]:
meat.head()

,Menu + Option,Unnamed: 1,Name,Lactose free?,Gluten Free?,Vegan?,Vegetarian?,Grab and Go?,Amount g,Product 1,...,Amount g.8,Product 9,Amount g.9,Product 10,Amount g.10,Product 11,Amount g.11,Product 12,Amount g.12,Product 13
1,NaN,Option 2,Roast Beef,NaN,x,NaN,NaN,NaN,210.0,Beef,...,40.0,Parmesan cheese,10.0,cream,50.0,Tomatoes,100.0,Zucchini,50.0,Red bell pepper
3,NaN,Option 4,Fish stikcs with vegetables,NaN,x,NaN,NaN,x,200.0,Fish sticks,...,10.0,garlic,20.0,canola oil,15.0,soy sauce,10.0,sambal,NaN,NaN
4,NaN,Option 5,Ham Sandich,NaN,NaN,NaN,NaN,x,200.0,bread,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,Option 7,Chicken Satay Sandwich,NaN,NaN,NaN,NaN,x,200.0,bread,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,Option 2,Sauteed Pork steam,NaN,NaN,NaN,NaN,NaN,210.0,Pork,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
recipe_dict = {}
for index, row in recipes.iterrows():
    recipe = {}
    rowiter = 0
    for rowiter in range(8, 34, 2):
        if row[rowiter] != None:
            recipe[row[rowiter+1]] = row[rowiter]
    recipe_dict[row['Name']] = recipe